# Facenet recreation
Google facenet recreation

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import itertools
from matplotlib import image
import glob as glob
from PIL import Image

import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchinfo import summary

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
  print("Using the GPU!")
else:
  print("WARNING: Could not find GPU! Using CPU only. If you want to enable GPU, please to go Edit > Notebook Settings > Hardware Accelerator and select GPU.")


# Build Dataset from facenet
using instructions from pytorch docs: https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
Reading pairs to ensure that muiltiple faces of a single person 


In [ ]:
class LabledFacesWild(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        label_dict = []
        with open(annotations_file) as f:
            for line in f:
                name, i1, i2 = line.split('\t')
                label_dict.append({
                    "label": name,
                    "img1": f"{name}/{i1:04d}.jpg",
                    "img2": f"{name}/{i2:04d}.jpg"
                })
        self.img_labels = pd.DataFrame.from_records(label_dict)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels) * 2

    def __getitem__(self, idx):
        pair_num = idx // 2
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[pair_num, (idx % 2) + 1])
        image = read_image(img_path)
        label = self.img_labels.iloc[pair_num, 0]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
# actually load data
num_train_pairs = 1100
num_test_pairs = 500
batch_size = 40
